# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [34]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

In [35]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [36]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [37]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

####################NEU################

In [38]:
print(dir(pt_dataset))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_configure', '_describe_component', '_irds_id', '_irds_ref', 'get_corpus', 'get_corpus_iter', 'get_corpus_lang', 'get_index', 'get_qrels', 'get_results', 'get_topics', 'get_topics_lang', 'get_topicsqrels', 'info_url', 'irds_ref']


In [39]:
# Assuming pt_dataset is already defined and has the get_corpus_iter() method
for i, doc in enumerate(pt_dataset.get_corpus_iter()):
    if i >= 10:
        break
    print(doc)

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 10/126958 [00:00<00:12, 9941.46it/s]

{'text': 'A Study on Word Similarity using Context Vector Models\n\n\n There is a need to measure word similarity when processing natural languages, especially when using generalization, classification, or example -based approaches. Usually, measures of similarity between two words are defined according to the distance between their semantic classes in a semantic taxonomy . The taxonomy approaches are more or less semantic -based that do not consider syntactic similarit ies. However, in real applications, both semantic and syntactic similarities are required and weighted differently. Word similarity based on context vectors is a mixture of syntactic and semantic similarit ies. In this paper, we propose using only syntactic related co-occurrences as context vectors and adopt information theoretic models to solve the problems of data sparseness and characteristic precision. The probabilistic distribution of co-occurrence context features is derived by parsing the contextual environment o

In [49]:
# Assuming pt_dataset is already defined and initialized

# Test get_corpus_iter
print("Testing get_corpus_iter()")
for i, doc in enumerate(pt_dataset.get_corpus_iter()):
    if i >= 10:  # Print the first 10 documents
        break
    print(doc)



Testing get_corpus_iter()


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 10/126958 [00:00<00:11, 11450.46it/s]

{'text': 'A Study on Word Similarity using Context Vector Models\n\n\n There is a need to measure word similarity when processing natural languages, especially when using generalization, classification, or example -based approaches. Usually, measures of similarity between two words are defined according to the distance between their semantic classes in a semantic taxonomy . The taxonomy approaches are more or less semantic -based that do not consider syntactic similarit ies. However, in real applications, both semantic and syntactic similarities are required and weighted differently. Word similarity based on context vectors is a mixture of syntactic and semantic similarit ies. In this paper, we propose using only syntactic related co-occurrences as context vectors and adopt information theoretic models to solve the problems of data sparseness and characteristic precision. The probabilistic distribution of co-occurrence context features is derived by parsing the contextual environment o

In [58]:
# Assuming pt_dataset is already defined and initialized

# Count the number of documents in the corpus
corpus_size = sum(1 for _ in pt_dataset.get_corpus_iter())

print(f"The corpus contains {corpus_size} documents.")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 56355.17it/s]

The corpus contains 126958 documents.


In [50]:
# Test get_qrels
print("Testing get_qrels()")
qrels = pt_dataset.get_qrels()
print(qrels)


Testing get_qrels()
     qid                                docno  label iteration
0      1    2005.ipm_journal-ir0volumeA41A1.7      1         0
1      1   2019.tois_journal-ir0volumeA37A1.2      1         0
2      1   2008.sigirconf_conference-2008.127      1         0
3      1    2015.ipm_journal-ir0volumeA51A5.7      0         0
4      1   2008.tois_journal-ir0volumeA27A1.1      0         0
...   ..                                  ...    ...       ...
2618  18  1985.jasis_journal-ir0volumeA36A3.9      0         0
2619  18      2010.wwwconf_conference-2010.11      1         0
2620  18       2011.ntcir_workshop-2011evia.3      0         0
2621  18    1988.ipm_journal-ir0volumeA24A3.1      0         0
2622  18         2010.airs_conference-2010.12      0         0

[2623 rows x 4 columns]


In [51]:
# Test get_topics
print("Testing get_topics()")
topics = pt_dataset.get_topics()
print(topics)

#############
# text, title are the same
# query: similar to text/title but all lowercase and without any special characters
# description: query formulated as a question --> Contains None values
# narrative: Concrete explanation of what the user is looking for

Testing get_topics()
There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
   qid                                      text  \
0    1  retrieval system improving effectiveness   
1    2  machine learning language identification   
2    3             social media detect self-harm   
3    4             stemming for arabic languages   
4    5            audio based animal recognition   
..  ..                                       ...   
63  65         information in different language   
64  66                  Abbreviations in queries   
65  67                  lemmatization algorithms   
66  68                  filter ad rich documents   
67  18     Advancements in Information Retrieval   

                                       title  \
0   retrieval system improving effectiveness   
1   machine learning language identification   
2              social media det

In [52]:
# Test info_url
print("Testing info_url()")
info_url = pt_dataset.info_url()
print(info_url)

Testing info_url()
https://ir-datasets.com/ir-lab-sose-2024.html#ir-lab-sose-2024/ir-acl-anthology-20240504-training


In [53]:
# Test irds_ref
print("Testing irds_ref()")
irds_ref = pt_dataset.irds_ref()
print(irds_ref)


Testing irds_ref()
Dataset(id='ir-lab-sose-2024/ir-acl-anthology-20240504-training', provides=['docs', 'queries', 'qrels'])


### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [54]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

### Step 4: Create the Run


In [55]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').tail(20)
# even with 'text' is shows 'query'??

First, we have a short look at the first three topics:


,qid,query
48,50,query optimization
49,51,cosine similarity vector
50,52,reverse indexing
51,53,index compression techniques
52,54,search engine optimization with query logs
53,55,bm25
54,56,what makes natural language processing natural
55,57,principle of a information retrieval indexing
56,58,architecture of web search engine
57,59,what is ahp


In [56]:
print('Now we do the retrieval...')
run = bm25(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...


Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conference-2004.47,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir0volumeA25A4.2,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir0volumeA41A5.11,2,14.144223,retrieval system improving effectiveness
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness
4,1,84876,2016.ntcir_conference-2016.90,4,13.947994,retrieval system improving effectiveness
5,1,82472,1998.sigirconf_conference-98.15,5,13.901647,retrieval system improving effectiveness
6,1,94415,2008.cikm_conference-2008.183,6,13.808208,retrieval system improving effectiveness
7,1,17496,O01-2005,7,13.749449,retrieval system improving effectiveness
8,1,82490,1998.sigirconf_conference-98.33,8,13.735541,retrieval system improving effectiveness
9,1,124801,2006.ipm_journal-ir0volumeA42A3.2,9,13.569263,retrieval system improving effectiveness


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [57]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
